List of python package to be installed:

1. numpy
2. pytorch
3. pandas
4. pycox
5. sklearn

In [ ]:
import pandas as pd
import torch

from dataset.syndata.syndata_v06 import DGP
from model.experiment import Experiment

Load dataset

In [ ]:
# minimum censored instances are necessary to conduct predictive performance comparison with other methods
synthetic = DGP(lambda_1 = 1.5, lambda_2 = 0.5)
df =  synthetic.generate_samples(N = 3000, random_state = 13, p_censor = 0.1)

Spcify each column in relation to the generative process. Each variable indicates the following:

1. t_cols: survival time
2. s_cols: (failure) event (0: right-censored, 1: death)
3. c_cols: (continuous) physiological measurements
4. x_cols: (binary) morbidity indicators
5. a_cols: (continuous) age
6. b_cols: (both continuous and discrete) personal background

In [ ]:
t_cols = ['durations'] 
s_cols = ['events']
c_cols = ['c']
b_cols = ['b']
a_cols = ['a']
x_cols = ['x']

Specify a set of columns, i.e. a set of all continuous variables except for survival time, that we normalise before the analysis.

In [ ]:
cols_to_std = ['c']

In [ ]:
directory = "syndata_v1/k_fold_cv/"

Define Experiment clase whose argument includes main dataset (df) and a set of columns defined above.

In [ ]:
experiment = Experiment(df, 
                t_cols = t_cols, s_cols = s_cols, c_cols = c_cols,
                x_cols = x_cols, a_cols = a_cols, b_cols = b_cols,
                       directory = directory)

Specify hyperparameters including

1. hidden_dim: # of neurons
2. lr: learning rate
3. n_epochs: # of epochs 
4. batch_size: batch size
5. device: "cpu", "mps" and "gpu"

Performance evaluations

In [ ]:
n_epochs = 1000
batch_size = 512
lr = 5e-3
hidden_dim = 64
alpha = 0.7
beta = 1.0
gamma = [1.0, 10.0]
z_dim = 2
k_peaks = 1
fold = 10

Train M4VAE and compare its predictive performance against other baseline methods, using K-fold cross-validation.

Key arguments:

return_baseline: $\textbf{standard Cox, DeepCox, Deep time-dependent Cox, DeepHit, DeSurv and SuMo-Net}$ 

return_metrics: $\textbf{c-index, brier score, negative binomial log-likelihood and log-likelihood}$, calibration score $\textbf{D-calibration}$, \textbf{silverman test} and \textbf{diptest}

In [ ]:
experiment.run_evaluations_by_fold(cols_to_std = cols_to_std, num_folds = fold, v_prop = 0.1, random_state = 13, 
                                    hidden_dim = hidden_dim, z_dim = z_dim, alpha = alpha, beta = beta, gamma = gamma, 
                                    lr = lr, n_epochs = n_epochs, batch_size = batch_size, logging_freq = 10, max_wait = 20, 
                                     device = 'cpu', return_baseline = True, verbose = False, k_peaks = k_peaks)

Present outcome that is comprised of
1. eval_dict: evaluation metrics described above
2. surv_dict: predictive survival curve inferred by each model and test set

In [ ]:
surv_dict = torch.load("result/" + directory + f"surv_dict_10_fold")
eval_dict = torch.load("result/" + directory + f"eval_dict_10_fold")

In [ ]:
import numpy as np

for metric in eval_dict.keys():
    print(f"\n{metric}")
    for model in eval_dict[metric].keys():
        mean = np.mean(eval_dict[metric][model]).round(3)
        print(f"{model}: {np.quantile(a = eval_dict[metric][model], q = [0.025, 0.975], method = 'closest_observation').round(3)}, Mean: {mean}")


In [ ]:
for metric in eval_dict.keys():
    print(f"\n{metric}")
    for model in eval_dict[metric].keys():
        mean = np.mean(eval_dict[metric][model]).round(3)
        std = np.std(eval_dict[metric][model]).round(3)
        print(f"{model}: {mean} $\pm$ {std}")

In [ ]:
import matplotlib.pylab as plt
import numpy as np

def show_pred_curve( i, surv_dict, figsize = (6, 4) ):
    """
        i: patient index
    """
    plt.figure(figsize = figsize)
    for key in surv_dict.keys():
        event = surv_dict['Test'].iloc[i, :]['events']
        time = surv_dict['Test'].iloc[i, :]['durations']
        t = np.round(time, 3)
        
        if key != "Test":
            plt.plot(surv_dict[key].iloc[:, i], label = key)
        
            
        plt.xlim([0, surv_dict['Test']['durations'].max()])
        plt.xlabel("Time-to-failure")
        plt.ylabel("Survival probability")
    
    plt.axvline(t, color = 'black', linestyle = 'dashed', label = f't = %.2f ' % t + f'| s = {event}')
    plt.legend()

Show predictive survival curves inferred by every model.

In [ ]:
show_pred_curve(6, surv_dict)

In [ ]:
show_pred_curve(50, surv_dict)

Present evaluation metrics

experiment.model_selections(cols_to_std = cols_to_std, hidden_dim = hidden_dim, lr = lr, 
                    n_epochs = n_epochs, batch_size = batch_size, device = device,
                           z_grid = [2, 10], alpha_grid = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 
                           beta_grid = [1.0])

It runs a set of benchmark models including $\textbf{standard Cox, DeepCox, Deep time-dependent Cox, DeepHit, DeSurv and SuMo-Net}$ to the dataset and save their predictive performance in terms of $\textbf{c-index, brier score, negative binomial log-likelihood and log-likelihood}$. 

In [ ]:
def summary_data( experiment ):
    obs, cens = experiment.df[experiment.events].value_counts()

    print(f"Observed: {obs} ({np.round(obs / (obs + cens), 2)} )")
    print(f"Censored: {cens}, ({np.round(cens / (obs + cens), 2)})")
    
    b_dim = len(experiment.bidx) + len(experiment.aidx)
    c_dim = experiment.c_dim
    x_dim = len(experiment.xidx)
    
    print(f"(Continuous) covariates: {c_dim}")
    print(f"(Binary) covariates: {x_dim}")
    print(f"Auxiliary covariates: {b_dim}")
    
    event_subset = experiment.df[experiment.df[experiment.events] == 1]
    t_mean, t_max = event_subset[experiment.durations].mean(), event_subset[experiment.durations].max()
    
    print(f"Event time / Mean: {np.round(t_mean, 1)}")
    print(f"Event time / Max: {np.round(t_max, 1)}")
    
    censor_subset = experiment.df[experiment.df[experiment.events] == 0]
    s_mean, s_max = censor_subset[experiment.durations].mean(), censor_subset[experiment.durations].max()
    
    print(f"Censoring time / Mean: {np.round(s_mean, 1)}")
    print(f"Censoring time / Max: {np.round(s_max, 1)}")

In [ ]:
summary_data(experiment)